# Домашнее задание седьмого урока.

Ваш менее опытный коллега не уверен, что его таблица retail действительно создалась и добавилась в базу данных. Напишите запрос, который создаст таблицу, если она еще не существует (**IF NOT EXISTS**), со следующими параметрами:

Название: **retail**

База: **test**
Столбцы:

- InvoiceNo – идентификатор заказа, строковое значение

- StockCode – идентификатор товара, строковое значение

- Description – описание товара, строковое значение

- Quantity – количество единиц товара, добавленных в корзину (особенность данных – значения могут быть отрицательными, 32 бита)

- InvoiceDate – дата и время создания заказа (транзакции), часовой пояс – 'Europe/London'

- UnitPrice – цена за единицу товара (особенность данных – значения могут быть отрицательными, точность – три знака после точки, 64 бита)

- CustomerID – идентификатор пользователя, целое положительное число, 32 бита.

- Country – страна, строковое значение. 

Движок: **MergeTree**

Сортировка: **InvoiceDate, CustomerID**
```sql
CREATE TABLE IF NOT EXISTS test.retail (
    InvoiceNo String,   
    StockCode  String,    
    Description  String,   
    Quantity  Int32,   
    InvoiceDate DateTime('Europe/London'),   
    UnitPrice Decimal64(3),    
    CustomerID Uint32,   
    Country String  
    )    
ENGINE = MergeTree   
ORDER BY (InvoiceDate, CustomerID)
```
---------------------
В следующих шагах используется **default.retail** – данные о транзакциях британского интернет-магазина в период с 1 декабря 2010 по 9 декабря 2011, где:

* InvoiceNo – номер транзакции
* StockCode – код товара
* Description – описание товара
* Quantity – количество единиц товара, добавленных в заказ
* InvoiceDate – дата транзакции 
* UnitPrice – цена за единицу товара
* CustomerID – id клиента
* Country – страна, где проживает клиент

**2. Для начала посмотрите, в каких странах имеется наибольшее число уникальных пользователей. Постройте барчарт, где:**

* по оси x – страна
* по оси y – число пользователей

**В качестве ответа укажите число уникальных пользователей из Великобритании.** 

Note: запятые в Redash отделяют каждые три знака в числе (тысячные доли, миллионные и так далее). Например, 2,535,966 - это 2535966. В ответ надо писать число без запятых.
```sql
SELECT COUNT(DISTINCT CustomerID) AS uniq_users,
       Country      
FROM default.retail
GROUP BY Country
```

![](https://i.ibb.co/VM27W8t/dash-rt-1.png)

Ответ: `3950`


**3. Следующая задача – посмотреть на динамику изменения числа активных пользователей в месяц в Великобритании, Австралии и Нидерландах. Полученная вами результирующая таблица должна иметь вид: страна - число уникальных пользователей за определённый месяц.**

* MAU (monthly active users) – число уникальных пользователей за месяц. Активные пользователи – те, кто сделал хотя бы один заказ за выбранный промежуток времени (месяц).

**В качестве ответа укажите наименьшее число пользователей за февраль 2011.** 

```sql
SELECT Country,
       COUNT(DISTINCT CustomerID) AS uniq_users      
FROM default.retail
WHERE
    toStartOfMonth(InvoiceDate) = '2011-02-01'    
GROUP BY Country
HAVING Country IN ('United Kingdom', 'Australia', 'Netherlands')
ORDER BY uniq_users ASC
```

![](https://i.ibb.co/dG208YD/dash-rt-2.png)


Ответ: `2`

**4. Как вы могли заметить, график с предыдущего шага получается не очень информативным для 2 из 3 стран. Давайте посмотрим на динамику изменения числа активных пользователей в месяц уже для всех стран, кроме Великобритании.**

**В каких месяцах число активных пользователей во Франции было больше, чем в Германии?**
```sql
SELECT Country,
       COUNT(DISTINCT CustomerID) AS uniq_users,       
       toStartOfMonth(InvoiceDate) AS month       
FROM default.retail
WHERE Country IN ('France', 'Germany')    
GROUP BY Country, month
```

![](https://i.ibb.co/S7MBRHd/dash-rt-3.png)


* **Май 2011**
* Всегда
* Июнь 2011
* **Сентябрь 2011**
* **Декабрь 2011**
* Декабрь 2010

**5. Теперь проанализируем сами заказы. Посчитайте среднюю сумму заказа (AOV – average order value) в каждой из стран.**

**Обратите внимание, что пользователь может добавить несколько одинаковых товаров в один заказ, и для подсчета суммы каждого заказа можно создать столбец TotalPrice, в котором будет храниться цена, которую пользователь заплатил, например, за три упаковки чая.**


InvoiceNo |	Description|Quantity |	UnitPrice	|TotalPrice
---------:|------------|--------:|------------:|:-------:
12345     |	     12345 |	    3|	       1.2 |	3.6 
12345     |	COOKIES    |	 1	 |         2.5 | 	2.5


**Далее для каждого заказа нужно просуммировать полученную колонку, а затем сгруппировать данные по странам и посчитать среднюю сумму заказа в каждой из них.**

**Визуализируйте результат. Из представленных вариантов ответа выберите страны, входящие в топ-3 по величине средней суммы заказа.**

NB! В таблице есть отрицательные значения Quantity, что соответствует отмененным заказам. Их в расчет AOV включать не нужно. 

```sql
SELECT
       Country,      
       AVG(TotalPrice) AS avg_total_price
FROM 
    (SELECT Country,    
            SUM(UnitPrice * Quantity) AS TotalPrice            
FROM default.retail
WHERE Quantity > 0
GROUP BY Country, InvoiceNo)
GROUP BY Country
ORDER BY AVG(TotalPrice) DESC
```

![](https://i.ibb.co/K03JcGQ/dash-rt-4.png)

* **Netherlands**
* **Australia**
* **Singapore**
* Sweden
* Japan
* United Kingdom


**6. Как изменялась средняя сумма заказа в разных странах по месяцам?**

**Предположим, нас интересует динамика в следующих странах: United Kingdom, Germany, France, Spain, Netherlands, Belgium, Switzerland, Portugal, Australia, USA. Визуализируйте результат и выберите верные утверждения.**

```sql
SELECT
    Country,   
    Month,    
    AVG(Price) AS AOV   
FROM (
    SELECT   
        InvoiceNo,      
        SUM(Quantity * UnitPrice) AS Price,     
        toStartOfMonth(InvoiceDate) AS Month,       
        Country       
    FROM default.retail   
    WHERE Quantity > 0 
    GROUP BY InvoiceNo, Country, Month   
    ORDER BY InvoiceNo   
    )  
WHERE Country In ('United Kingdom', 'Germany', 'France', 'Spain', 'Netherlands', 'Belgium', 'Switzerland', 'Portugal', 'Australia', 'USA')
GROUP BY Country, Month
```

![](https://i.ibb.co/XY7Tz4N/dash-rt-5.png)

* Аномальный рост среднего чека произошел в июле 2011 среди пользователей из Австралии
* **Аномальный рост среднего чека произошел в августе 2011 среди пользователей из Австралии**
* **Самые большие значения среднего чека наблюдаются в Австралии и Нидерландах.**
* Второй по размеру средний чек – в Германии
* **Примерно до октября средний чек США постепенно рос**
* Средний чек в Великобритании значительно выше, чем во всех других странах

**7. Сколько товаров пользователи обычно добавляют в корзину? Посчитайте среднее количество товаров, добавленных в корзину, с разбивкой по странам.**

**В качестве ответа укажите среднее число товаров в заказе среди пользователей в Канаде (Canada). Разделитель - точка.**

```sql
SELECT Country,
       avg(count_stock) AS avg_count       
FROM (
    SELECT Country,    
            SUM(Quantity) as count_stock         
    FROM default.retail   
    WHERE Quantity > 0    
    GROUP BY Country, InvoiceNo)    
GROUP BY Country
ORDER BY avg_count DESC
```

![](https://i.ibb.co/zfwZrCK/dash-rt-6.png)

Ответ: `460.50`


**8. Возможно, результат на предыдущем шаге показался вам странным, особенно если соотнести средний размер корзины с числом уникальных пользователей в некоторых странах.** 

**Посмотрим на Нидерланды (Netherlands) более подробно. Сгруппируйте данные по пользователям и посмотрите, кто купил наибольшее число товаров. В ответе укажите идентификатор данного пользователя.**

```sql
SELECT CustomerID,
       SUM(Quantity) AS sum_quantity       
FROM default.retail
WHERE Quantity > 0 AND Country = 'Netherlands'
GROUP BY CustomerID
ORDER BY sum_quantity DESC
```

![](https://i.ibb.co/NS3WX5P/dash-rt-7.png)

Ответ: `14646`

**9. Один из пользователей добавил к себе в корзину почти 200 тысяч товаров. Не кажется ли это подозрительным? Изучите, что именно он покупает, посмотрите на количество товаров в каждом заказе, итоговые суммы. Подумайте, кто или что это может быть 🤔**

**Когда подумаете, просто введите идентификатор данного пользователя еще раз в поле ниже.**

Ответ: `14646`

**10. Задание на построение дашборда**

**Следующая задача: подготовить дашборд в Redash (состоящий из более чем четырх графиков). Есть несколько основных вариантов, и в обоих случаях вы можете дополнить его любыми графиками, которые кажутся вам интересными и полезными:**

**Построить дашборд для отдела продаж. Вывести основные показатели: например, выручку по странам, дням, топ товаров, и другие графики, которые вы считаете необходимыми для отражения текущего положения дел.**

**Самостоятельно поставить себе задачу :)**  

**В поле ниже добавьте следующую информацию:**

* Краткое описание и основные выводы (если вы выбрали свою тему, то желательно описать более подробно)

* Ссылку на ваш дашборд в Redash (формат: https://redash.lab.karpov.courses/dashboards/....)